
# 0.2 Python 必备工具回顾（快速复习 · 实战版）

本 Notebook 使用 `scikit-learn` 自带的 **Breast Cancer**（乳腺癌分类）与 **Diabetes**（糖尿病进展回归）数据集，演示：  
- `pandas` 的常用数据操作  
- 快速可视化（直方图、箱线图、相关性热图）  
- 从因果视角对 EDA 的注意事项（**EDA ≠ 因果**）



## 0.2.1 `pandas` 数据操作（基于乳腺癌数据集）

> 数据集：`sklearn.datasets.load_breast_cancer()`  
> 练习内容：DataFrame 转换、筛选、分组统计、交叉表与缺失/类型检查。


In [ ]:

import pandas as pd
from sklearn.datasets import load_breast_cancer

# 1) 载入并转换为 DataFrame
data = load_breast_cancer()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.Series(data.target, name='target')  # 1=malignant(恶性), 0=benign(良性)
df = pd.concat([X, y], axis=1)

# 2) 快速查看结构与类型
print("形状:", df.shape)
print(df.head(3))
print("\n信息(info)：")
print(df.info())

# 3) 缺失值检查（该数据集通常没有缺失，这里演示写法）
print("\n缺失值统计：")
print(df.isnull().sum().sort_values(ascending=False).head())

# 4) 简单筛选：示例——筛选“恶性”样本中 mean radius > 20 的病例
df_sel = df[(df['target'] == 1) & (df['mean radius'] > 20)]
print("\n筛选后样本量:", df_sel.shape[0])

# 5) 分组统计：按 target（良/恶性）查看若干关键特征的均值
cols_focus = ['mean radius', 'mean texture', 'mean symmetry']
group_mean = df.groupby('target')[cols_focus].mean()
print("\n按 target 分组的均值：")
print(group_mean)

# 6) 交叉表（构造一个示例离散特征）
df['radius_bin'] = (df['mean radius'] > df['mean radius'].median()).astype(int)
crosstab = pd.crosstab(index=df['target'], columns=df['radius_bin'], margins=True)
print("\n交叉表(target vs radius_bin)：")
print(crosstab)

# 7) 描述性统计
print("\n描述性统计（部分列）：")
print(df[cols_focus + ['target']].describe())



## 0.2.2 可视化（直方图 / 箱线图 / 相关性热图）

> 工具：`matplotlib` + `seaborn`  
> 提示：可视化展示的是**关联**与**分布**，不能直接得出因果结论。


In [ ]:

import matplotlib.pyplot as plt
import seaborn as sns

# 1) 直方图：mean radius 分布（按 target 分层）
plt.figure()
sns.histplot(data=df, x='mean radius', hue='target', kde=True, element='step', stat='density')
plt.title('Distribution of Mean Radius by Target (0=Benign, 1=Malignant)')
plt.show()

# 2) 箱线图：不同 target 的 mean radius
plt.figure()
sns.boxplot(data=df, x='target', y='mean radius')
plt.title('Mean Radius by Target')
plt.show()

# 3) 相关性热图（选部分特征以避免图过密）
subset_cols = ['mean radius','mean texture','mean perimeter','mean area','mean smoothness']
plt.figure()
sns.heatmap(df[subset_cols].corr(), annot=True, fmt='.2f')
plt.title('Correlation Heatmap (Selected Features)')
plt.show()



## 0.2.3 补充练习：Diabetes 回归数据（连续结局）

> 数据集：`sklearn.datasets.load_diabetes()`（目标为疾病进展的连续指标）。


In [ ]:

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_diabetes

# 1) 载入并转换
ddata = load_diabetes()
Xd = pd.DataFrame(ddata.data, columns=ddata.feature_names)
yd = pd.Series(ddata.target, name='disease_progression')  # 连续结局
df_diab = pd.concat([Xd, yd], axis=1)

print("形状:", df_diab.shape)
print(df_diab.head())

# 2) 结局变量直方图
plt.figure()
sns.histplot(df_diab['disease_progression'], kde=True)
plt.title('Distribution of Diabetes Progression')
plt.show()

# 3) 特征与结局的关系（举例：bmi 与 y）
plt.figure()
sns.scatterplot(data=df_diab, x='bmi', y='disease_progression')
plt.title('BMI vs Disease Progression')
plt.show()

# 4) 相关性热图（选取几个常见特征）
sel = ['bmi','bp','s1','s5','disease_progression']
plt.figure()
sns.heatmap(df_diab[sel].corr(), annot=True, fmt='.2f')
plt.title('Correlation (Selected Features)')
plt.show()



## 0.2.4 小结与注意事项（因果视角）
- 以上步骤是 EDA（探索性分析），用于理解数据结构、分布和潜在混杂候选，但**不提供因果结论**。  
- 若要推断“某治疗是否导致结局变化”，必须明确**暴露/治疗**与**结局**的时间顺序，并控制**混杂**（下一阶段会涉及 PSM/IPTW/DR 等方法）。  
- 真实医学数据常有缺失与异常值；需要制定合理的缺失处理与质量控制策略。
